# Street processing


The goal of this file is to
* Process the 1836 dataset (finding / working with duplicate streets)
* Define which streets belong to only one or both datasets
* Have one final dataset containing all georeferenced streets

In [ ]:
# Imports
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import geopandas as gpd
import contextily as cx
import matplotlib.pyplot as plt
from shapely.ops import linemerge, Point
#from shapely.errors import ShapelyDeprecationWarning
import warnings
#warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 
#from importnb import Notebook
from preprocessing import preprocess
from collections import Counter
from paris_methods import duplicate_processing, duplicate_final, assign_gridnumber, translate_geopoints, create_grid
import pyproj

In [ ]:
# Import shapefiles
Openparis = gpd.read_file("data/voie.zip", encoding = 'utf-8')
Vasserot = gpd.read_file("data/vasserot.zip")

# Set right EPSG for Geodata
Openparis = Openparis.to_crs(epsg=3857)
Vasserot = Vasserot.to_crs(epsg=3857)

#change mistakes in streetnames
mask = Vasserot.loc[:,"NOM_ENTIER"] == "Rue Lafayette"
Vasserot.loc[mask,["NOM","NOM_ENTIER"]] = ["la Fayette", "Rue la Fayette"]


## Preprocess streets with leas code

In [ ]:
Openparis = preprocess(Openparis, "l_longmin")
Vasserot = preprocess(Vasserot, "NOM_ENTIER")

# TODO Rename those streets
Vasserot["voie"] = Vasserot["NOM_ENTIER_prep"]
Openparis["voie"] = Openparis["l_longmin_prep"]

# Remove empty lines
Vasserot = Vasserot.dropna(subset=["voie"])

# Add year
Vasserot = Vasserot.assign(year= [[1836]]*len(Vasserot))
Openparis = Openparis.assign(year= [[2022]]*len(Openparis))

# create buffer around streets, important for merging duplicate streets
buffer = 100
Vasserot["buffer"] = Vasserot["geometry"].apply(lambda x: x.buffer(buffer))
Openparis["buffer"] = Openparis["geometry"].apply(lambda x: x.buffer(buffer))



# Find all duplicates
Duplicates = Vasserot[Vasserot.duplicated(subset=['voie'], keep=False)].sort_values("voie")
Unique = Vasserot[~Vasserot.duplicated(subset=['voie'], keep=False)].sort_values("voie")



## Process 1836 streets

### Merge duplicates if streets are close

In [ ]:
DuplicatesProcessed = duplicate_processing(Duplicates, "voie")

# Sanity check if all streetnames are in the newly created dataframe
#len(Duplicates["voie"].unique()) == DuplicatesProcessed["voie"].unique()))

### Visualizing Nr. of unique streets before and after processing

In [ ]:
freqs = Counter(Duplicates["voie"].value_counts())
plt.bar(freqs.keys(), freqs.values(), width = 0.9)
freqs = Counter(DuplicatesProcessed["voie"].value_counts())
plt.bar(freqs.keys(), freqs.values(), width = 0.9)
plt.show()

print("Duplicates before: ", sum(Duplicates["voie"].value_counts()>1))
print("Duplicates after: ", sum(DuplicatesProcessed["voie"].value_counts()>1))

In [ ]:
StillDuplicates = DuplicatesProcessed[DuplicatesProcessed.duplicated(subset=['voie'], keep=False)].sort_values("voie")
NewlyUnique = DuplicatesProcessed[~DuplicatesProcessed.duplicated(subset=['voie'], keep=False)].sort_values("voie")
# Adding newly unique streets
Unique = pd.concat([Unique, NewlyUnique])

#### Visualizing Streets that are still duplicates

In [ ]:
# Needed in order to visualize results
StillDuplicates = StillDuplicates.drop(columns=["buffer"])

In [ ]:
# Select Street to visualize
a = StillDuplicates["voie"].value_counts()[StillDuplicates["voie"].value_counts()>=2]
Trueduplicates = a.index
mask = StillDuplicates["voie"] == Trueduplicates[8]
# Visualize
#StillDuplicates[mask].explore()
StillDuplicates.explore()

## Create dataset containing old and new streets


#### Preparing Vasserot and Openparis dataframe for final comparison

In [ ]:
# Unique from Vasserot DF
Unique = Unique.iloc[:,[0,2,3,6,8,14,16,17,18]]
Unique = Unique.rename(columns={"ROWID":"rowid", "NOM_ENTIER":"streetname","TYPE":"type","ARTICLE":"article","NOM":"name", "voie":"streetname_prep"})
Unique = Unique.assign(matching = [[]] * len(Unique))

In [ ]:
# Uniques from Openparis DF
Openparis = Openparis.iloc[:,[2,3,4,5,6,15,17,18,19]]
Openparis = Openparis.rename(columns={"l_longmin": "streetname","c_desi":"type","c_liaison":"article","l_voie":"name","l_courtmin":"streetname_short","voie":"streetname_prep"})
# assign random rowid to Openparis data because they dont have them
Openparis = Openparis.assign(rowid = np.random.randint(7000, 200000, size=len(Openparis)))
Openparis = Openparis.assign(matching = [[]] * len(Openparis))

Merged = pd.concat([Unique, Openparis])


In [ ]:
# Check if streets with same name are at same location 
MergedProcessed = duplicate_final(Merged, "streetname_prep")
MergedProcessed = preprocess(MergedProcessed, "name")

# Create Datasets containing duplicates and unique streets
FinalDuplicates = MergedProcessed[MergedProcessed.duplicated(subset=['streetname_prep'], keep=False)].sort_values("streetname_prep")
FinalUnique= MergedProcessed[~MergedProcessed.duplicated(subset=['streetname_prep'], keep=False)].sort_values("streetname_prep")

# Assign right classes to data
FinalUnique = FinalUnique.convert_dtypes()
FinalDuplicates = FinalDuplicates.convert_dtypes()

### Visualize results in the final dataset

In [ ]:
FinalDuplicates= FinalDuplicates.drop(columns=["buffer"])
FinalDuplicates.explore()

In [ ]:
FinalUnique["geometry"].explore()

## Saving Dataframes

In [ ]:

FinalUnique.to_pickle("data/FinalUnique.pkl")
FinalDuplicates.to_pickle("data/FinalDuplicate.pkl")
